In [6]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM


squad = load_dataset("squad")
contexts = set()
for item in squad['validation']:
    contexts.add(item['context'])

tokenizer_ag = AutoTokenizer.from_pretrained("models/t5-small_asnwergen_overfit/checkpoint-3000")
model_ag = AutoModelForSeq2SeqLM.from_pretrained("models/t5-small_asnwergen_overfit/checkpoint-3000")

contexts = list(contexts)[:100]
counter = 0
for context in contexts:
    tokenized_in_ag = tokenizer_ag(
            context,
            max_length=256,
            truncation="only_first",
            padding="max_length",
            return_tensors="pt",
        )

    input_ids_ag = tokenized_in_ag["input_ids"][0]


    output = model_ag.generate(
        input_ids_ag.reshape(1, -1), 
        num_beams=20, 
        # max_length=10,
        decoder_start_token_id=tokenizer_ag.pad_token_id,
        num_return_sequences=10
    )
    answers = tokenizer_ag.batch_decode(output, skip_special_tokens=False)
    answers = [ans.strip('<pad>')[1:-3] for ans in answers]
    for answer in answers:
        # if more than 10 words increment counter
        if len(answer.split(" ")) > 10:
            counter += 1
            

Found cached dataset squad (/storagenfs/m.tolloso/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 357.08it/s]


In [8]:
counter

143